In [ ]:
import os

from dotenv import load_dotenv

In [2]:
# set env vars
load_dotenv("./.env")
os.environ["LANGSMITH_PROJECT"] = "tool_calling_extraction"

In [3]:
# define schema
from typing import Optional

from pydantic import BaseModel, Field


class Computer(BaseModel):
    """Specifications of any computing device"""

    model: Optional[str] = Field(description="The specific model number or name of the computing device.")
    storage: Optional[int] = Field(description="How many GBs the computng device has as the total storage.")
    num_cpu: Optional[int] = Field(description="The number of physical CPUs the device has.", ge=1)
    gpus: Optional[list[str]] = Field(description="The graphics processing units the comuting device has.")

In [ ]:
# design prompt
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert information extraction machine. Only extract relevant, accurat and true information from the input. If you don't know the value of an attribute you are asked to extract, you can return None. Don't make up information you don't know.",
        ),
        ("human", "{text}"),
    ]
)

In [6]:
# set up structured output
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-5", model_provider="openai")

structured_llm = llm.with_structured_output(schema=Computer)

In [ ]:
# run inference
text = "I'm looking to upgrade my intel dual core proecssor. It's nvidia gpu is getting bottlenecked by the weak igpu. It's an old Lenovo thinkstation."
prompt = prompt_template.invoke(input={"text": text})
structured_llm.invoke(prompt)

Computer(model=None, storage=None, num_cpu=1, gpus=['NVIDIA GPU', 'Integrated GPU'])